In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/Colab Notebooks/lab05/Lab 5.zip" -d "content/"

Archive:  /content/drive/MyDrive/Colab Notebooks/lab05/Lab 5.zip
   creating: content/Lab 5/
  inflating: content/Lab 5/GOOG.csv  
  inflating: content/Lab 5/IMDB Dataset.csv  
  inflating: content/Lab 5/Q1.ipynb  
  inflating: content/Lab 5/Q2.ipynb  
  inflating: content/Lab 5/Q3.ipynb  


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import re

In [4]:
# 1. Load and Preprocess the Dataset
def load_data(file_path):
    # Load the dataset (e.g., IMDB movie reviews dataset)
    df = pd.read_csv(file_path, engine='python', on_bad_lines='skip')  # Using 'python' engine and skipping bad lines
    df.dropna(inplace=True)  # Drop any rows with missing values
    return df['review'], df['sentiment']  # Assuming 'review' and 'sentiment' columns

In [5]:
# Clean the text
def clean_text(text):
    # Remove unwanted characters, numbers, and symbols
    text = re.sub(r"[^A-Za-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [6]:
# Tokenize and Pad Sequences
def preprocess_text(reviews, max_words=5000, max_len=200):
    reviews = [clean_text(review) for review in reviews]  # Clean the reviews
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(reviews)
    sequences = tokenizer.texts_to_sequences(reviews)
    padded_sequences = pad_sequences(sequences, maxlen=max_len)
    return padded_sequences, tokenizer



In [7]:
# Encode Sentiments
def encode_labels(sentiments):
    sentiments = sentiments.map({'positive': 1, 'negative': 0}).values
    return sentiments

In [8]:
import os
print(os.listdir("content/Lab 5"))

['IMDB Dataset.csv', 'Q2.ipynb', 'GOOG.csv', 'Q3.ipynb', 'Q1.ipynb']


In [9]:
file_path = 'content/Lab 5/IMDB Dataset.csv'
reviews, sentiments = load_data(file_path)


In [10]:
# Preprocess Text Data
max_words = 5000  # Consider the top 5000 words
max_len = 200  # Pad or truncate reviews to 200 words
X, tokenizer = preprocess_text(reviews, max_words=max_words, max_len=max_len)

In [11]:
# Encode Sentiments (positive -> 1, negative -> 0)
y = encode_labels(sentiments)

# Split into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# 2. Define the LSTM Model
model = Sequential()

# Modify the embedding dimensions and experiment with LSTM configurations ---
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))  # <-- Modify 'output_dim'
model.add(Bidirectional(LSTM(units=64, return_sequences=False)))  # <-- Experiment with 'units' and add Dropout if necessary

model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Train the Model
#  Modify 'epochs' and 'batch_size' to see how they impact training time and model accuracy ---
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=1)  # <-- Experiment with 'epochs' and 'batch_size'


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 170s 135ms/step - accuracy: 0.7591 - loss: 0.4796 - val_accuracy: 0.8577 - val_loss: 0.3426
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 213s 143ms/step - accuracy: 0.8946 - loss: 0.2673 - val_accuracy: 0.8799 - val_loss: 0.2973
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 191s 135ms/step - accuracy: 0.9185 - loss: 0.2121 - val_accuracy: 0.8870 - val_loss: 0.2809
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 203s 136ms/step - accuracy: 0.9352 - loss: 0.1727 - val_accuracy: 0.8874 - val_loss: 0.2853
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 211s 143ms/step - accuracy: 0.9506 - loss: 0.1349 - val_accuracy: 0.8760 - val_loss: 0.3402
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 201s 143ms/step - accuracy: 0.9598 - loss: 0.1138 - val_accuracy: 0.8760 - val_loss: 0.3631
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 202s 143ms/step - accuracy: 0.9747 - loss: 0.0752 - val_accuracy: 0.8652 - val_loss: 0.4513
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 202s 143ms/step - ac

In [13]:
# 4. Evaluate the Model
y_pred = (model.predict(X_test) > 0.5).astype("int32")


313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step


In [14]:
# Calculate Accuracy and F1-Score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1-Score: {f1:.4f}')


#  Analyze the accuracy and F1-score. Consider modifying the model architecture or hyperparameters to improve performance ---

Accuracy: 0.8750
F1-Score: 0.8745
